In [1]:
# Local imports
import os
import glob
import re
import sys
import argparse

# Third-party imports
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.cm as mpl_cm
import matplotlib
import cartopy.crs as ccrs
import iris
import iris.coord_categorisation as coord_cat
import iris.plot as iplt
import scipy
import pdb
import datetime
import iris.quickplot as qplt

# Import dictionaries and functions from my module
import dictionaries as dic
import functions as fnc

In [ ]:
# Set up the variables
model = "HadGEM3-GC31-MM"
variable = "psl"
region = "global"
season = "DJFM"
forecast_range = "2-9"
start_year = "1960"
end_year = "2022"
obs_path = dic.obs_path
models = dic.psl_full_models
base_dir = dic.base_dir_skm_pro
output_dir = dic.plots_dir_canari

# Set up the matched variable
tas_models = dic.tas_models
match_var_tas = "tas"

In [ ]:
# Process the observations
obs_anomaly = fnc.read_obs(variable, region, forecast_range,
                            season, obs_path, start_year, end_year)

In [ ]:
# Load and process the model data
datasets = fnc.load_data(dic.base_dir_skm_pro, models, variable, region,
                            forecast_range, season)

# Process the model data
model_anomaly, _ = fnc.process_data(datasets, variable)

In [ ]:
# Make sure that the model and obs have the same time period
model_anomaly = fnc.constrain_years(model_anomaly, models)

# Remove years containing nans from the observations and align the time periods
# for the observations and model
obs_anomaly, model_anomaly = fnc.remove_years_with_nans(obs_anomaly, model_anomaly, models)

In [ ]:
# Calculate the NAO index
obs_nao, model_nao = fnc.calculate_nao_index_and_plot(obs_anomaly, model_anomaly, models, 
                                                        variable, season, forecast_range, 
                                                            output_dir, plot_graphics=False)

In [7]:
# Test the NAO rescaling function
rescaled_nao, ensemble_mean_nao, ensemble_members_nao = fnc.rescale_nao(obs_nao, model_nao, models, season,
                                                                        forecast_range, output_dir, lagged=False)

<xarray.DataArray 'filled-7c9d14396f574f261f7d21efde2390bd' (time: 53)>
dask.array<sub, shape=(53,), dtype=float64, chunksize=(8,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1966-12-31 1967-12-31 ... 2018-12-31


In [ ]:
# Perform the NAO matching for tas
matched_tas_ensemble_mean = fnc.nao_matching_other_var(rescaled_nao, model_nao,
                                                        models, match_var_tas, dic.base_dir_skm_pro, 
                                                            tas_models, obs_path, region,
                                                                season, forecast_range, start_year, 
                                                                    end_year, output_dir, lagged=False,
                                                                        no_subset_members=20)

In [ ]:
# Now we want to calculate the spatial correlations between the matched variable and its observations
# First we need to process the observations
obs_tas = fnc.read_obs(match_var_tas, region, forecast_range,
                        season, obs_path, start_year, end_year)

In [ ]:
# Now we can calculate the spatial correlations
rfield, pfield, \
obs_lons_converted, lons_converted = fnc.calculate_spatial_correlations(obs_tas,
                                                                            matched_tas_ensemble_mean, 
                                                                                tas_models, 
                                                                                    match_var_tas)